- 전처리 미포함, 학습

In [5]:
import pandas as pd

review_jungrang_ori = pd.read_csv('C:/Users/user/Desktop/saltlux_project/ML_DL/preprocess/JungrangTotal.csv', index_col=0)
review_jungrang_ori = review_jungrang_ori[['name', 'nickname', 'content']].reset_index(drop=True)

In [6]:
review_jungrang_ori['content'] = review_jungrang_ori['content'].str.replace('\n', ' ')
review_jungrang_ori = review_jungrang_ori.reset_index(drop=True)
review_jungrang_ori

,name,nickname,content
0,365mc모인이비인후과의원,Wiseburge David,사가정 모인 이비인후과 항상 만원이다
1,365mc모인이비인후과의원,Dk9,좋아요
2,365mc모인이비인후과의원,귀한집딸z,좋아요
3,365mc모인이비인후과의원,꽃길걷는중임,항상 친절한 진료 감사합니다~
4,365mc모인이비인후과의원,막둥,ㅇㅇ
...,...,...,...
5970,효치과의원,fpvocalist,굳
5971,효치과의원,소보루39,좋아요
5972,효치과의원,1vvovv1,친절하십니다 과잉진료 없음
5973,효치과의원,버럭아저씨,친절해요..


In [7]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.functional import softmax
import torch

# 'content' 컬럼의 데이터를 리스트로 추출
texts = review_jungrang_ori['content'].tolist()

# BERT 모델 및 토크나이저 불러오기
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# 결과를 저장할 리스트 초기화
results = []

# 임계값 설정
threshold = 0.4

# 각 텍스트에 대해 확률 계산 및 결과 저장
for text in texts:
    tokens = tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        logits = model(**tokens).logits

    probs = softmax(logits, dim=1).squeeze().tolist()
    result_str = f"긍정 확률: {probs[1]:.2%}, 부정 확률: {probs[0]:.2%}"
    results.append({'Text': text, 'Result': result_str})

# 결과를 DataFrame으로 변환
results_df = pd.DataFrame(results)

# 결과를 CSV 파일로 저장
results_df.to_csv('../results/bert_results_ori_0.4prob.csv', index=False)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
